<a href="https://colab.research.google.com/github/tyrantech/Chinook-/blob/main/chinook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
df = files.upload()

Saving Chinook_Sqlite.sqlite to Chinook_Sqlite.sqlite


In [ ]:
import sqlite3
conn = sqlite3.connect("Chinook_Sqlite.sqlite")
curse = conn.cursor()
curse.execute("SELECT name FROM sqlite_master where type='table';")
tables = curse.fetchall()
print(tables)


[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]


In [ ]:

import pandas as pd
q="""WITH InvoiceTotals AS (
    SELECT
        i.InvoiceId,
        i.CustomerId,
        i.BillingCity,
        IFNULL(i.BillingState,'State is Unknown') AS billingState,
        i.BillingCountry,
		CASE strftime("%m",i.InvoiceDate)
		WHEN "01" THEN "January"
		WHEN "02" THEN "Febuary"
		WHEN "03" THEN "March"
		WHEN "04" THEN "April"
		WHEN "05" THEN "May"
		WHEN "06" THEN "June"
		WHEN "07" THEN "July"
		WHEN "08" THEN "August"
		WHEN "09" THEN "September"
		WHEN "10" THEN "October"
		WHEN "11" THEN "November"
		WHEN "12" THEN "December"
		END AS monthBucket,

        i.Total AS InvoiceTotal
    FROM Invoice i
),
topSales AS (
    SELECT
        e.EmployeeId,
        e.FirstName,
        e.LastName,
        SUM(i.Total) AS salesMade
    FROM Employee e
    JOIN Customer c ON e.EmployeeId = c.SupportRepId
    JOIN Invoice i ON c.CustomerId = i.CustomerId
    GROUP BY e.EmployeeId
    ORDER BY salesMade DESC
)

SELECT
    c.FirstName,
    c.LastName,
    it.BillingCountry,
    it.BillingCity,
    it.billingState,
	ts.FirstName,ts.LastName,ts.salesMade,
    ar.Name AS ArtistName,
    a.Title AS AlbumName,
    g.Name AS GenreName,
    SUM(il.Quantity * il.UnitPrice) AS trackSales,
    SUM(it.InvoiceTotal) AS invoiceSales,it.monthBucket
FROM InvoiceLine il
JOIN InvoiceTotals it ON il.InvoiceId = it.InvoiceId
JOIN Customer c ON it.CustomerId = c.CustomerId
JOIN topSales ts ON c.SupportRepId = ts.EmployeeId
JOIN Track t ON il.TrackId = t.TrackId
JOIN Album a ON t.AlbumId = a.AlbumId
JOIN Artist ar ON a.ArtistId = ar.ArtistId
JOIN Genre g ON t.GenreId = g.GenreId
GROUP BY
    c.FirstName,
    c.LastName,
    it.BillingCountry,
    it.BillingCity,
    it.billingState,
    ar.Name,
    a.Title,
    g.Name,it.monthBucket
ORDER BY
    trackSales DESC,
    invoiceSales DESC,ts.salesMade DESC,it.monthBucket DESC;

"""
from sqlalchemy import create_engine
engine = create_engine('sqlite:///Chinook_Sqlite.sqlite')
df = pd.read_sql(q,engine)
df.to_csv("sales_report.csv",index=False)
print("Saved as sales_report.csv")
from google.colab import files
files.download("sales_report.csv")
df.head()

Saved as sales_report.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,FirstName,LastName,BillingCountry,BillingCity,billingState,FirstName,LastName,salesMade,ArtistName,AlbumName,GenreName,trackSales,invoiceSales,monthBucket
0,Hugh,O'Reilly,Ireland,Dublin,Dublin,Jane,Peacock,833.04,Lost,"Lost, Season 2",TV Shows,11.94,131.16,April
1,Helena,Holý,Czech Republic,Prague,State is Unknown,Steve,Johnson,720.16,Lost,"Lost, Season 1",TV Shows,7.96,103.44,November
2,Richard,Cunningham,USA,Fort Worth,TX,Margaret,Park,775.40,Lost,"Lost, Season 3",Drama,7.96,95.44,August
3,Julia,Barnett,USA,Salt Lake City,UT,Steve,Johnson,720.16,Battlestar Galactica (Classic),"Battlestar Galactica (Classic), Season 1",Sci Fi & Fantasy,7.96,47.76,September
4,Frank,Ralston,USA,Chicago,IL,Jane,Peacock,833.04,The Office,"The Office, Season 3",Comedy,7.96,31.84,September
